In [526]:
import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pylab as plt
import numpy as np


tf.keras.backend.clear_session()  # For easy reset of notebook state.

im_size = 224

In [527]:
dir_train = "dataset/Train"
train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    dir_train,
    image_size=(im_size, im_size),
    batch_size=32,

)

Found 3916 files belonging to 4 classes.


In [528]:
dir_validation = "dataset/Validation"
validation_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    dir_validation,
    image_size=(im_size, im_size),
    batch_size=32,

)

Found 1314 files belonging to 4 classes.


In [529]:
dir_test = "dataset/Test"
test_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    dir_test,
    image_size=(im_size, im_size),
    batch_size=32,

)

Found 1312 files belonging to 4 classes.


In [530]:
type(test_dataset)

tensorflow.python.data.ops.dataset_ops.BatchDataset

In [531]:
class_names = train_dataset.class_names
num_classes = len(class_names)

In [532]:
#mobilenet_v2 = "https://tfhub.dev/google/supcon/resnet_v1_200/imagenet/classification/1"

In [533]:
#mobile_net_layers = hub.KerasLayer(mobilenet_v2, input_shape=(224,224,3))

In [534]:
#mobile_net_layers.trainable = False

In [535]:
'''
model = tf.keras.Sequential([
    mobile_net_layers,
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(num_classes,activation='softmax')
])
'''

model = tf.keras.Sequential([
    tf.keras.layers.Rescaling(1./255, input_shape=(im_size, im_size  , 3)),
  tf.keras.layers.Conv2D(16, 3, padding='same', activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(32, 3, padding='same', activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(num_classes)
])


In [536]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [537]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling (Rescaling)       (None, 224, 224, 3)       0         
                                                                 
 conv2d (Conv2D)             (None, 224, 224, 16)      448       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 112, 112, 16)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 112, 112, 32)      4640      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 56, 56, 32)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 56, 56, 64)        1

In [ ]:
history = model.fit(train_dataset, epochs=10, validation_data=validation_dataset)

Epoch 1/10
 49/123 [==========>...................] - ETA: 51s - loss: 1.2405 - accuracy: 0.4662

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(10)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
  from skimage import io

sunflower_path = "1111.jpg"
ig = io.imread(sunflower_path)
plt.imshow(ig)
plt.show()

img = tf.keras.utils.load_img(
    sunflower_path, target_size= (im_size, im_size)
)
img_array = tf.keras.utils.img_to_array(img)
img_array = tf.expand_dims(img_array, 0) # Create a batchN
predictions = model.predict(img_array)
score = tf.nn.softmax(predictions[0])

print(
    "This image most likely belongs to {} with a {:.2f} percent confidence."
    .format(class_names[np.argmax(score)], 100 * np.max(score))
)

In [ ]:
model.evaluate(test_dataset)

In [ ]:
model.save('model_pruba_2_platos.h5')

In [ ]:
IMAGE_SIZE = [224, 224]

In [ ]:
from tensorflow.keras.models import load_model
model = load_model("model_pruba_2_platos.h5")

In [ ]:
from skimage import io
im_size = 256
sunflower_path = "1111.jpg"
ig = io.imread(sunflower_path)
plt.imshow(ig)
plt.show()

img = tf.keras.utils.load_img(
    sunflower_path, target_size= (im_size, im_size)
)
img_array = tf.keras.utils.img_to_array(img)
img_array = tf.expand_dims(img_array, 0) # Create a batchN
predictions = model.predict(img_array)
score = tf.nn.softmax(predictions[0])

print(
    "This image most likely belongs to {} with a {:.2f} percent confidence."
    .format(class_names[np.argmax(score)], 100 * np.max(score))
)